## Anpassen von Funktionen und Daten mit Unsicherheit

Wir betrachten hier das Anpassen von Funktionen and Daten, bei denen die y-Daten fehlerbehaftet sind.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
position = np.array([10, 14, 23, 40, 46, 58, 63, 73, 98, 110])
potential = np.array([0.53, 0.67, 0.94, 1.27, 0.95, 1.49, 2.06, 2.30, 2.85, 3.06])
delta = np.array([0.05, 0.05, 0.05, 0.15, 0.05, 0.15, 0.15, 0.15, 0.15, 0.15]) 

### Anapssen mittels Polygonfit

Die Funktion `np.polyfit(x,y,deg=1,w=weights)` passt ein Polygon ersten Grades (also eine Gerade) an die Daten an.

Die Unsicherheiten werden hier als Vektor von sogenannten *Gewichte* (Weights) an die Funktion übergeben. Anschaulich bedeutet dies, dass ein Datenpunkt mit **kleinerer** Unsicherheit ein **höheres** Gewicht hat.

In der Literatur wird als Gewichtung normalerweise die Inverse der Varianz verwendet. [Weiterführender Link](https://de.wikipedia.org/wiki/Verallgemeinerte_Kleinste-Quadrate-Sch%C3%A4tzung#Gewichtete_kleinste_Quadrate_(GKQ)

Die Python Funktion `np.polyfit` verwendet eine andere Konvention der Gewichte: Die Gewichte sind die Inverse der Standardabweichung.

$$W = \frac{1}{\delta}$$

In [3]:
weights = 1 / delta
params, cov_mat = np.polyfit(position, potential, deg=1, w=weights, cov=True)
print(f'Fitparameter: a={params[1]:.3f}, b={params[0]:.3f}')
print(f'Unsicherheiten: sigma_a={np.sqrt(cov_mat[1,1]):.3f}, sigma_b={np.sqrt(cov_mat[0,0]):.3f}')

Fitparameter: a=0.289, b=0.022
Unsicherheiten: sigma_a=0.124, sigma_b=0.003


### Anpassen mittels curve_fit

Das python Paket `scipy` stellt die Funktion `scipy.optimize.curve_fit(model, x, y, sigma=sigma)` zur Verfügung.

Mit dieser Routinge kann eine allgemeine Funktion, die mittels `model` definiert ist, anpassen. Hier wird statt den Gewichten die Standardabweichung mittels des Parameters `sigma` übergeben.



In [4]:
def linear_model(x, a, b):
    return a * x + b 

params_curve_fit, cov_curve_fit = scipy.optimize.curve_fit(linear_model, position, potential, sigma=delta)
print(f'Fitparameter: a={params_curve_fit[1]:.3f}, b={params_curve_fit[0]:.3f}')
print(f'Unsicherheiten: sigma_a={np.sqrt(cov_curve_fit[1,1]):.3f}, sigma_b={np.sqrt(cov_curve_fit[0,0]):.3f}')

Fitparameter: a=0.289, b=0.022
Unsicherheiten: sigma_a=0.124, sigma_b=0.003


### Anmerkungen

Abschließend soll gesagt werden, dass eine mathematisch korrekte Behandlung von Unsicherheiten weit über die Inhalte des Grundpraktikums hinausgeht. Zum Beispiel hängen die mit obigen Methoden erhaltenen Unsicherheiten der Parameter nur von der relativen Unsicherheit der Daten ab. Wenn die Unsicherheiten der Daten mit einem konstanten Faktor multipliziert werden, ändert sich an dern Resultaten nichts:

In [5]:
new_weights = 10 *weights
params, cov_mat = np.polyfit(position, potential, deg=1, w=new_weights, cov=True)
print(f'Fitparameter 10 faches sigma_y: a={params[1]:.3f}, b={params[0]:.3f}')
print(f'Unsicherheiten 10 faches sigma_y: sigma_a={np.sqrt(cov_mat[1,1]):.3f}, sigma_b={np.sqrt(cov_mat[0,0]):.3f}')

Fitparameter 10 faches sigma_y: a=0.289, b=0.022
Unsicherheiten 10 faches sigma_y: sigma_a=0.124, sigma_b=0.003


In [6]:
params_curve_fit, cov_curve_fit = scipy.optimize.curve_fit(linear_model, position, potential, sigma=10*delta)
print(f'Fitparameter 10 faches sigma_y: a={params_curve_fit[1]:.3f}, b={params_curve_fit[0]:.3f}')
print(f'Unsicherheiten 10 faches sigma_y: sigma_a={np.sqrt(cov_curve_fit[1,1]):.3f}, sigma_b={np.sqrt(cov_curve_fit[0,0]):.3f}')

Fitparameter 10 faches sigma_y: a=0.289, b=0.022
Unsicherheiten 10 faches sigma_y: sigma_a=0.124, sigma_b=0.003


Die Funktion `curve_fit` kann absolute Unsicherheiten mittels des Parameters `absolute_sigma=True` berücksichtigen:

In [7]:
params_curve_fit, cov_curve_fit = scipy.optimize.curve_fit(linear_model, position, potential, sigma=10*delta, absolute_sigma=True)
print(f'Fitparameter 10 faches sigma_y absolut: a={params_curve_fit[1]:.3f}, b={params_curve_fit[0]:.3f}')
print(f'Unsicherheiten 10 faches sigma_y absolut: sigma_a={np.sqrt(cov_curve_fit[1,1]):.3f}, sigma_b={np.sqrt(cov_curve_fit[0,0]):.3f}')

Fitparameter 10 faches sigma_y absolut: a=0.289, b=0.022
Unsicherheiten 10 faches sigma_y absolut: sigma_a=0.377, sigma_b=0.010
